In [1]:
import pandas as pd
import traceback

In [2]:
#EXTRACT
dfs = []
for i in range (1,101):
    for j in range(1,3):
        try:
            df = pd.read_excel(f"../data/isruc-sleep/subgroupI/{i}/{i}_{j}.xlsx", usecols="A:I",)
            if not "spo2" in [str(col).lower() for col in df.columns]:
                                    print(f"../data/isruc-sleep/subgroupI/{i}/{i}_{j}.xlsx has no header.")
                                    df = pd.read_excel(f"../data/isruc-sleep/subgroupI/{i}/{i}_{j}.xlsx", usecols="A:I", header=None)
            df["patient_id"] = i
            df["score_id"]= j
            dfs.append(df)
        except Exception as e:
            print(f"Error parsing {i}: {e}")
            traceback.print_exc()

../data/isruc-sleep/subgroupI/11/11_1.xlsx has no header.
../data/isruc-sleep/subgroupI/11/11_2.xlsx has no header.
../data/isruc-sleep/subgroupI/33/33_2.xlsx has no header.
../data/isruc-sleep/subgroupI/54/54_1.xlsx has no header.
../data/isruc-sleep/subgroupI/54/54_2.xlsx has no header.


In [3]:
# TRANSFORM
def transform(df):
    df = df.copy()
    df.columns=["epoch","stage","spo2","hr","events","bpos","txln","txex","technote", "patient_id","score_id"]

    #no point keeping points if scorer didnt score
    df.dropna(how="any", subset=['epoch','stage'], inplace=True)
    df.dropna(how="all", inplace=True)
    df.loc[df.technote.isna(), "technote"]=""
    df.loc[df.events.isna(),'events']= ""
    df['events'] = df.apply(lambda r: r['events'] + " ".join(r['hr'].split(" ")[1:]) if isinstance(r['hr'],str) else r['events'], axis=1)
    df['hr'] = df.hr.apply(lambda r: float(r.split(" ")[0]) if isinstance(r, str) else float(r))
    df.epoch = df.epoch.astype("int32")
    return df

In [4]:
dfs = [transform(df) for df in dfs]

In [5]:
df = pd.concat(dfs)

In [6]:
df

,epoch,stage,spo2,hr,events,bpos,txln,txex,technote,patient_id,score_id
0,1,W,0.0,69.0,"L Out,MChg",R,0.0,0.0,,1,1
1,2,W,97.0,67.0,,R,0.0,0.0,,1,1
2,3,W,97.0,60.0,,R,0.0,0.0,,1,1
3,4,W,97.0,60.0,,R,0.0,0.0,,1,1
4,5,W,97.0,64.0,,R,0.0,0.0,,1,1
...,...,...,...,...,...,...,...,...,...,...,...
856,857,R,95.0,69.0,Awake,L,0,0,,100,2
857,858,R,95.0,69.0,Awake,L,0,0,,100,2
858,859,R,95.0,55.0,,L,0,0,,100,2
859,860,R,95.0,53.0,,L,0,0,,100,2


In [7]:
df.to_csv("../data/isruc-sleep/subgroupI/scoring.")

In [8]:
studyI = pd.read_excel(f"../data/isruc-sleep/subgroupI/Details_subgroup_I_Submission.xlsx", header=2,usecols="A:S")

In [9]:
studyI.columns=[x.lower() for x in studyI.columns]

In [10]:
studyI['age']=pd.to_numeric(studyI.age,errors="coerce")
studyI['date of recording'] = pd.to_datetime(studyI['date of recording'])

In [11]:
studyI.head()

,subject,age,sex,diagnosis,other problems,epoches,w%,n1%,n2%,n3%,rem%,date of recording,w%.1,n1%.1,n2%.1,n3%.1,rem%.1,medication,eeg alterations
0,1,64.0,M,SAOS,Depression,880,30.00,8.30,22.05,26.25,13.41,2009-05-18,30.91,4.77,24.09,26.25,13.98,SSRI; trazodona,NaN
1,2,52.0,M,SAOS,Restless leg syndrome,964,25.41,11.93,35.79,16.29,10.58,2009-08-06,21.99,11.93,44.40,11.10,10.58,NaN,NaN
2,3,38.0,M,REM Sleep Behaviour Disorder,PLMS,943,14.00,17.50,26.09,18.35,24.07,2009-05-21,12.73,4.45,39.24,15.69,27.89,Risperidona; Tegretol,NaN
3,4,27.0,M,SRVAS,Epilepsy,963,2.91,6.75,44.24,22.22,23.88,2009-06-18,2.70,3.43,43.61,25.65,24.61,NaN,Paroxistic activity left fronto-parietal
4,5,58.0,F,SAOS,Insomnia,875,33.83,12.34,30.29,18.74,4.80,2009-05-25,35.66,4.69,35.66,18.86,5.14,BZD,NaN


In [12]:
studyI.to_csv("../data/isruc-sleep/subgroupI/scoring_summary.csv")

#from visbrain.gui import Sleep

In [13]:
#import os

#from visbrain.gui import Sleep
#from visbrain.io import download_file, path_to_visbrain_data